In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
from scipy.special import boxcox1p

# processing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

# model
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.metrics import mean_squared_error

from scipy import stats
from scipy.stats import norm, skew

warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^.*LAPACK bug 0038.*")

In [2]:
# load training data split train_x and train_y
train_pd = pd.read_csv("all/train.csv")

# remove outlier from data analysis
# drop outliers
# query = train_pd[(train_pd['GrLivArea'] > 4000) & (train_pd['SalePrice'] < 200000)]
# train_pd = train_pd.drop(query.index.values)

train_x = train_pd.drop(['SalePrice'], axis=1)
train_x['is_train'] = 1
# train_y = train_pd['SalePrice']
#log transform the target:
train_y = np.log1p(train_pd["SalePrice"])
# print(train_y)
# print(np.log1p(train_y["SalePrice"]))
test_pd = pd.read_csv("all/test.csv")
test_pd['is_train'] = 0
print(train_x.shape)
print(train_y.shape)
print(test_pd.shape)

(1460, 81)
(1460,)
(1459, 81)


In [3]:
# concat train and test

all_data = pd.concat((train_x, test_pd)).reset_index(drop=True)
print(all_data.shape)

(2919, 81)


In [4]:
# remove ID
all_data = all_data.drop(['Id'], axis=1)
print(all_data.shape)

(2919, 80)


In [5]:
# nan data
NAN = [(col, all_data[col].isna().mean()*100) for col in all_data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
print(NAN)

NAN = NAN[NAN.percentage > 50]
NAN.sort_values("percentage", ascending=False)
print(NAN)

      column_name  percentage
0      MSSubClass    0.000000
1        MSZoning    0.137033
2     LotFrontage   16.649538
3         LotArea    0.000000
4          Street    0.000000
..            ...         ...
75         MoSold    0.000000
76         YrSold    0.000000
77       SaleType    0.034258
78  SaleCondition    0.000000
79       is_train    0.000000

[80 rows x 2 columns]
    column_name  percentage
5         Alley   93.216855
71       PoolQC   99.657417
72        Fence   80.438506
73  MiscFeature   96.402878


In [6]:
# There are 4 features has more than 80 percent nan value
# drop these columns
all_data = all_data.drop(['Alley', 'PoolQC', 'Fence','MiscFeature'], axis=1)
print(all_data.shape)

(2919, 76)


In [7]:
# split features into object_columns and numerical_columns
object_columns_df = all_data.select_dtypes(include=['object'])
numerical_columns_df = all_data.select_dtypes(exclude=['object'])
print(numerical_columns_df.isna().sum())
print(object_columns_df.isna().sum())

MSSubClass         0
LotFrontage      486
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea        23
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       2
BsmtHalfBath       2
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt      159
GarageCars         1
GarageArea         1
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
is_train           0
dtype: int64
MSZoning            4
Street              0
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2  

In [8]:
# fill lotfrontage with mean value corespond to overall quality
lot_not_nan = numerical_columns_df[numerical_columns_df['LotFrontage'].notna()]
lot_mean_map = dict(lot_not_nan.groupby('OverallQual').mean()['LotFrontage'])
all_data['LotFrontage'] = all_data['LotFrontage'].fillna(all_data['OverallQual'].map(lot_mean_map))
print(all_data['LotFrontage'].isna().sum())

0


In [9]:
# MasVnrArea nan means no Masonry veneer area
# just fill with 0
all_data['MasVnrArea'] = all_data['MasVnrArea'].fillna(0)
print(all_data['MasVnrArea'].isna().sum())

0


In [10]:
# BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF with 1 nan only. fill with 0
all_data['BsmtFinSF1'] = all_data['BsmtFinSF1'].fillna(0)
all_data['BsmtFinSF2'] = all_data['BsmtFinSF2'].fillna(0)
all_data['BsmtUnfSF'] = all_data['BsmtUnfSF'].fillna(0)
all_data['TotalBsmtSF'] = all_data['TotalBsmtSF'].fillna(0)
print(all_data['BsmtFinSF1'].isna().sum(), all_data['BsmtFinSF2'].isna().sum(),
     all_data['BsmtUnfSF'].isna().sum(), all_data['TotalBsmtSF'].isna().sum())

0 0 0 0


In [11]:
# BsmtFullBath, BsmtHalfBath means basement bathrooms, value can be 0
all_data['BsmtFullBath'] = all_data['BsmtFullBath'].fillna(0)
all_data['BsmtHalfBath'] = all_data['BsmtHalfBath'].fillna(0)
print(all_data['BsmtFullBath'].isna().sum(), all_data['BsmtHalfBath'].isna().sum())

0 0


In [12]:
# fill GarageYrBlt with average yearbuilt+3
all_data['GarageYrBlt'] = all_data['GarageYrBlt'].fillna(all_data['YearBuilt'] + 3)
print(all_data['GarageYrBlt'].isna().sum())

0


In [13]:
# GarageCars, GarageArea can be 0, fill with 0
all_data['GarageCars'] = all_data['GarageCars'].fillna(0)
all_data['GarageArea'] = all_data['GarageArea'].fillna(0)
print(all_data['GarageCars'].isna().sum(), all_data['GarageArea'].isna().sum())

0 0


In [14]:
# MasVnrType has None value, fill with None
all_data['MasVnrType'] = all_data['MasVnrType'].fillna('None')
print(all_data['MasVnrType'].isna().sum())

0


In [15]:
# BsmtQual BsmtCond BsmtExposure BsmtFinType1 BsmtFinType2 GarageType GarageFinish GarageQual GarageCond FireplaceQu have NA value, fill with NA
fill_na_col = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'FireplaceQu']
all_data[fill_na_col] = all_data[fill_na_col].fillna('NA')
print(all_data[fill_na_col].isna().sum())

BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
FireplaceQu     0
dtype: int64


In [16]:
# fill remain nan feature with common value
unfilled = all_data.columns[all_data.isna().any()].tolist()
all_data[unfilled] = all_data[unfilled].fillna(all_data[unfilled].mode().iloc[0])
print(all_data[unfilled].isna().sum())

MSZoning       0
Utilities      0
Exterior1st    0
Exterior2nd    0
Electrical     0
KitchenQual    0
Functional     0
SaleType       0
dtype: int64


In [17]:
print(all_data.shape)

(2919, 76)


In [18]:
# transform some feature to numerical
level_map  = {
    'Ex':5,
    'Gd':4,
    'TA':3,
    'Fa':2,
    'Po':1,
    'NA':0,
    'Y':1,
    'N':0,
    'Reg':3,
    'IR1':2,
    'IR2':1,
    'IR3':0,
    "Unf" : 1,
    "LwQ": 2,
    "Rec" : 3,
    "BLQ" : 4,
    "ALQ" : 5,
    "GLQ" : 6
}
# assign value 
all_data['LotShape'] = all_data['LotShape'].map(level_map)
all_data['ExterQual'] = all_data['ExterQual'].map(level_map)
all_data['ExterCond'] = all_data['ExterCond'].map(level_map)
all_data['BsmtQual'] = all_data['BsmtQual'].map(level_map)
all_data['BsmtCond'] = all_data['BsmtCond'].map(level_map)
all_data['BsmtFinType1'] = all_data['BsmtFinType1'].map(level_map)
all_data['BsmtFinType2'] = all_data['BsmtFinType2'].map(level_map)
all_data['HeatingQC'] = all_data['HeatingQC'].map(level_map)
all_data['CentralAir'] = all_data['CentralAir'].map(level_map)
all_data['KitchenQual'] = all_data['KitchenQual'].map(level_map)
all_data['FireplaceQu'] = all_data['FireplaceQu'].map(level_map)
all_data['GarageQual'] = all_data['GarageQual'].map(level_map)
all_data['GarageCond'] = all_data['GarageCond'].map(level_map)

print(all_data.shape)

(2919, 76)


In [19]:
# # transfer skewed features
# numerical_data = all_data.select_dtypes(exclude=['object'])
# skewValue = numerical_data.skew().sort_values(ascending=False)
# large_skew = skewValue[abs(skewValue) > 0.75]
# lamda = 0.15
# for feature in large_skew.index:
#     all_data[feature] = boxcox1p(all_data[feature], lamda)


In [20]:
# build house age feature
house_age = all_data['YrSold'] - all_data['YearBuilt']
all_data['house_age'] = house_age
print(all_data.shape)

(2919, 77)


In [21]:
# one_hot other features
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2919, 234)


In [22]:
all_data['YrBltAndRemod']=all_data['YearBuilt']+all_data['YearRemodAdd']
all_data['TotalSF']=all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

all_data['Total_sqr_footage'] = (all_data['BsmtFinSF1'] + all_data['BsmtFinSF2'] +
                                 all_data['1stFlrSF'] + all_data['2ndFlrSF'])

all_data['Total_Bathrooms'] = (all_data['FullBath'] + (0.5 * all_data['HalfBath']) +
                               all_data['BsmtFullBath'] + (0.5 * all_data['BsmtHalfBath']))

all_data['Total_porch_sf'] = (all_data['OpenPorchSF'] + all_data['3SsnPorch'] +
                              all_data['EnclosedPorch'] + all_data['ScreenPorch'] +
                              all_data['WoodDeckSF'])

# simplified features
all_data['haspool'] = all_data['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
all_data['has2ndfloor'] = all_data['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
all_data['hasgarage'] = all_data['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
all_data['hasbsmt'] = all_data['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
all_data['hasfireplace'] = all_data['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)
all_data['qual_square'] = all_data['OverallQual'].pow(2)
all_data['qual_cube'] = all_data['OverallQual'].pow(3)

print(all_data.shape)

(2919, 246)


In [23]:
train_x = all_data[all_data['is_train'] == 1]
test_x = all_data[all_data['is_train'] == 0]
train_x = train_x.drop(['is_train'], axis=1)
test_x = test_x.drop(['is_train'], axis=1)
print(train_x.shape)
print(test_x.shape)

(1460, 245)
(1459, 245)
